**Task: FIFO Warm‑Up**

In [2]:
# Show results after each step for the user's FIFO warm-up
q = []  # start empty
print("Start:", q)

# enqueue A
q.append('A')
print("After enqueue A:", q)

# enqueue B
q.append('B')
print("After enqueue B:", q)

# enqueue C
q.append('C')
print("After enqueue C:", q)

# dequeue operations
out1 = q.pop(0)  # dequeue -> 'A'
print("After dequeue ->", out1, "| Queue:", q)

out2 = q.pop(0)  # dequeue -> 'B'
print("After dequeue ->", out2, "| Queue:", q)

out3 = q.pop(0)  # dequeue -> 'C'
print("After dequeue ->", out3, "| Queue:", q)

print('Removed order:', out1, out2, out3)


Start: []
After enqueue A: ['A']
After enqueue B: ['A', 'B']
After enqueue C: ['A', 'B', 'C']
After dequeue -> A | Queue: ['B', 'C']
After dequeue -> B | Queue: ['C']
After dequeue -> C | Queue: []
Removed order: A B C


**Task: Teaching Queue**

In [4]:
# Teaching Queue (list-backed) with step-by-step outputs
class Queue:
    def __init__(self):
        self.items = []                 # internal list to hold elements
        print("Init ->", self.items)

    def is_empty(self):
        return len(self.items) == 0     # empty if length == 0

    def enqueue(self, x):
        self.items.append(x)            # append at end (rear)
        print(f"enqueue({x}) ->", self.items)

    def dequeue(self):
        if self.is_empty():
            print("dequeue() -> Underflow (None) |", self.items)
            return None                 # signal underflow
        val = self.items.pop(0)         # remove & return element at index 0 (front)
        print(f"dequeue() -> {val} |", self.items)
        return val

    def front(self):
        if self.is_empty():
            print("front() -> None |", self.items)
            return None
        print("front() ->", self.items[0], "|", self.items)
        return self.items[0]            # peek front without removing


# quick test with step-by-step tracing
q = Queue()
q.enqueue(10)
q.enqueue(20)
_ = q.dequeue()     # expect 10
_ = q.front()       # expect 20
print('is_empty ->', q.is_empty())  # expect False


Init -> []
enqueue(10) -> [10]
enqueue(20) -> [10, 20]
dequeue() -> 10 | [20]
front() -> 20 | [20]
is_empty -> False


**Task: Fixed-Size Array Queue**

In [12]:
# ---- Fixed-size Array Queue ---
class ArrayQueue:
    def __init__(self, size):
        self.size = size                 # total capacity
        self.a = [None] * size           # fixed-size storage
        self.front = 0                   # index of current front
        self.rear = -1                   # index of last filled position
        self.count = 0                   # number of current elements

    def is_empty(self):
        return self.count == 0

    def is_full(self):
        return self.count == self.size

    def enqueue(self, x):
        if self.is_full():
            print("Overflow")
            return False
        self.rear += 1                   # move right
        self.a[self.rear] = x            # place new element
        self.count += 1
        return True

    def dequeue(self):
        if self.is_empty():
            print("Underflow")
            return None
        val = self.a[self.front]         # read front
        self.front += 1                  # move right
        self.count -= 1
        return val

    def front_val(self):
        if self.is_empty():
            return None
        return self.a[self.front]


# demo: show waste behavior
aq = ArrayQueue(5)
aq.enqueue(10)
aq.enqueue(20)
aq.enqueue(30)
print('dequeue ->', aq.dequeue())        # remove 10
print('dequeue ->', aq.dequeue())        # remove 20
print('front   ->', aq.front_val())      # expect 30
print('state   ->', 'count =', aq.count, 'front =', aq.front, 'rear =', aq.rear)
print('active  ->', aq.a[aq.front:aq.rear + 1])  # logical active window


dequeue -> 10
dequeue -> 20
front   -> 30
state   -> count = 1 front = 2 rear = 2
active  -> [30]


**Task: Circular Queue**

In [11]:
# ---- Circular Queue with modulo wrap-around ---
class CircularQueue:
    def __init__(self, size):
        self.size = size
        self.a = [None] * size
        self.front = 0
        self.rear = -1
        self.count = 0

    def is_empty(self):
        return self.count == 0

    def is_full(self):
        return self.count == self.size

    def enqueue(self, x):
        if self.is_full():
            print("Overflow")
            return False
        self.rear = (self.rear + 1) % self.size
        self.a[self.rear] = x
        self.count += 1
        return True

    def dequeue(self):
        if self.is_empty():
            print("Underflow")
            return None
        val = self.a[self.front]
        self.front = (self.front + 1) % self.size
        self.count -= 1
        return val

    def front_val(self):
        if self.is_empty():
            return None
        return self.a[self.front]

    def to_list(self):
        # Return logical order of elements from front, length = count
        res = []
        idx = self.front
        for _ in range(self.count):
            res.append(self.a[idx])
            idx = (idx + 1) % self.size
        return res


# ---- Demo: wrap-around behavior ----
cq = CircularQueue(5)

for x in [10, 20, 30, 40]:
    cq.enqueue(x)

print('start   ->', cq.to_list())         # [10, 20, 30, 40]
print('dequeue ->', cq.dequeue())         # remove 10
print('dequeue ->', cq.dequeue())         # remove 20
print('mid     ->', cq.to_list())         # [30, 40]

# should wrap when needed
cq.enqueue(50)
cq.enqueue(60)

print('after   ->', cq.to_list())         # [30, 40, 50, 60]
print('front   ->', cq.front_val())
print('state   ->', 'front =', cq.front, 'rear =', cq.rear, 'count =', cq.count)


start   -> [10, 20, 30, 40]
dequeue -> 10
dequeue -> 20
mid     -> [30, 40]
after   -> [30, 40, 50, 60]
front   -> 30
state   -> front = 2 rear = 0 count = 4


**Task: Mini‑Project: Printer Jobs**

In [17]:
# ---- Printer Job Simulation ---
def run_printer_sim(jobs, capacity=8):
    q = CircularQueue(capacity)
    print("Incoming jobs:", jobs)
    for j in jobs:
        if not q.enqueue(j):
            print("Queue full, job dropped:", j)   # simple handling
    serviced = []
    while not q.is_empty():
        serviced.append(q.dequeue())
    print("Serviced order:", serviced)
    return serviced

# demo
jobs = ["J1", "J2", "J3", "J4", "J5"]
serviced = run_printer_sim(jobs, capacity=4)


Incoming jobs: ['J1', 'J2', 'J3', 'J4', 'J5']
Overflow
Queue full, job dropped: J5
Serviced order: ['J1', 'J2', 'J3', 'J4']
